In [ ]:
from sklearn.preprocessing import OneHotEncoder
from utils_public import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tqdm import tqdm, trange
import seaborn as sns
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.metrics import r2_score
def plot_and_r2(preds_train, preds_test, ratings_train, ratings_test, advisor): 
    #Calculates 
    plt.figure(figsize=(6,3))
    plt.scatter(ratings_train, preds_train, label='Train Set Preds', s=3, c = "#BBBBBB") #train set in gray
    plt.scatter(ratings_test, preds_test, label='Test Set Preds', s=5, c = "#DC267F") #test set in magenta
    plt.plot([0,1], [0,1], label="Target", linewidth=3, c="k") #target line in black

    #Set axis labels and title
    plt.xlabel("Actual Rating")
    plt.ylabel("Predicted Rating")
    plt.title(f"Advisor {advisor} Predictions")

    #Turn off top and right spines
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.legend() #Display legend
    plt.show() #Show plot

    #Calculate R2 score for train and test sets
    print(f"Advisor {advisor} Train Set R2 score: {r2_score(ratings_train, preds_train)}") 
    print(f"Advisor {advisor} Test Set R2 score: {r2_score(ratings_test, preds_test)}")


In [ ]:
# Load data and run the model
grids = load_grids()
ratings = np.load("datasets/scores.npy")
ratings_df = pd.DataFrame(ratings, columns=["Wellness", "Tax", "Transportation", "Business"])

In [ ]:
import random

num_iterations = 20  # how many new grids to create per original grid
num_cell_to_replace = 1  # how many cells to modify in each new grid

new_grids = []
new_ratings = []

for _ in range(num_iterations):
    for grid in grids:
        # Generate modified versions of the grid
        new_grid = grid.copy()
        for _ in range(num_cell_to_replace):
            # Select random cell coordinates
            x, y = random.randint(0, 6), random.randint(0, 6)
            current_val = grid[x, y]
            # Choose a different random value for replacement
            new_val = random.choice([v for v in range(5) if v != current_val])
            new_grid[x, y] = new_val
        new_grids.append(new_grid)
        new_ratings.append([np.nan] * 4)

# Convert lists to numpy arrays
new_grids = np.array(new_grids)
new_ratings = np.array(new_ratings)

# Combine with existing data
all_grids = np.vstack((grids, new_grids))
all_ratings = np.vstack((ratings, new_ratings))

# Remove duplicates
unique_grids, unique_indices = np.unique(all_grids, axis=0, return_index=True)
unique_ratings = all_ratings[unique_indices]

print(f"Number of unique grids: {unique_grids.shape[0]}")


In [ ]:
# Define model architecture for 7x7 grid prediction
def create_grid_cnn(input_shape=(7, 7, 5)):
    """
    Create a CNN model optimized for 7x7 grid prediction.

    Architecture features:
    - Small kernels (2x2, 3x3) to capture local patterns
    - Residual connections to help with gradient flow
    - Batch normalization for stable training
    - Dropout for regularization
    - Appropriate padding to maintain spatial information
    """
    inputs = layers.Input(shape=input_shape)

    # First convolutional block
    x = layers.Conv2D(32, kernel_size=3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x1 = x  # Store for residual connection (will project to match channels when needed)

    # Second convolutional block
    x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)

    # Third convolutional block with residual connection
    x = layers.Conv2D(64, kernel_size=3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Project the earlier x1 (32 channels) to match 64 channels before adding
    x1_proj = layers.Conv2D(64, kernel_size=1, padding='same')(x1)
    x1_proj = layers.BatchNormalization()(x1_proj)

    x = layers.Add()([x, x1_proj])  # Residual connection (now channel shapes match)
    x = layers.Activation('relu')(x)

    # Fourth convolutional block
    x = layers.Conv2D(128, kernel_size=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.3)(x)

    # Global pooling instead of flattening
    x = layers.GlobalAveragePooling2D()(x)

    # Dense layers
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation='sigmoid')(x)
    x = layers.BatchNormalization()(x)

    # Output layer
    outputs = layers.Dense(1, activation='linear')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model


all_predictions = [] #empty list to hold predictions
histories = []  # Store training histories for each advisor
for advisor in range(4): #loop over four advisors
    print(f"\nTraining model for Advisor {advisor}")
    grids_subset, ratings_subset = select_rated_subset(grids, ratings[:,advisor]) #gets subset of the dataset 
    X_train, X_validation, Y_train, Y_validation = train_test_split(grids_subset, ratings_subset, test_size=.2, random_state=42)

    height, width = 7, 7
    num_pixel_classes = 5  # pixels take values 0 to 4
    # # One-hot encoding of categorical pixel values
    X_train_encoded = tf.one_hot(X_train, depth=num_pixel_classes)  # Shape: (4000, 7, 7, 5)
    X_validation_encoded = tf.one_hot(X_validation, depth=num_pixel_classes)  # Shape: (4000, 7, 7, 5)

    # Create and compile model
    model = create_grid_cnn(input_shape=(height, width, num_pixel_classes))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    # Print model summary
    model.summary()

    # Train the model with early stopping
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Train the model and capture history
    history = model.fit(
        X_train_encoded, 
        Y_train, 
        batch_size=32, 
        epochs=100,  # Increased epochs since we have early stopping
        validation_data=(X_validation_encoded, Y_validation),
        callbacks=[early_stopping],
        verbose=1
    )
    histories.append(history)

    # #predict
    preds_train = model.predict(X_train_encoded) #predict on the train set
    preds_test = model.predict(X_validation_encoded) #predict on the test set

    plot_and_r2(preds_train, preds_test, Y_train, Y_validation, advisor)

    grids_encoded = tf.one_hot(unique_grids, depth=num_pixel_classes)  # Shape: (X, 7, 7, 5)
    # Merge predictions with actual ratings
    predictions = model.predict(grids_encoded) #predict on the train set

    mask = np.where(~np.isnan(ratings[:,advisor])) #get the indices of the rated grids
    predictions[mask, 0] = ratings[:, advisor][mask]  # assign to 2D predictions
    all_predictions.append(predictions) #append predictions
    all_predictions = [np.squeeze(a) for a in all_predictions]

In [ ]:
# Plot convergence curves for all advisors
plt.figure(figsize=(15, 10))

# Plot MSE Loss
plt.subplot(2, 1, 1)
for idx, history in enumerate(histories):
    plt.plot(history.history['loss'], label=f'Advisor {idx} Training Loss (MSE)')
    plt.plot(history.history['val_loss'], label=f'Advisor {idx} Validation Loss (MSE)', linestyle='--')
plt.title('Model Loss (MSE) over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

# Plot MAE
plt.subplot(2, 1, 2)
for idx, history in enumerate(histories):
    plt.plot(history.history['mae'], label=f'Advisor {idx} Training MAE')
    plt.plot(history.history['val_mae'], label=f'Advisor {idx} Validation MAE', linestyle='--')
plt.title('Model MAE over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

plt.tight_layout()
plt.show()

# Print final metrics for each advisor
for idx, history in enumerate(histories):
    print(f"\nAdvisor {idx} final metrics:")
    print(f"Training Loss (MSE): {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss (MSE): {history.history['val_loss'][-1]:.4f}")
    print(f"Training MAE: {history.history['mae'][-1]:.4f}")
    print(f"Validation MAE: {history.history['val_mae'][-1]:.4f}")

In [ ]:
# mask = np.where(~np.isnan(ratings[:,advisor])) #get the indices of the rated grids
# # predictions = predictions.reshape(-1, predictions.shape[0])
# predictions[mask, 0] = ratings[:, advisor][mask]  # assign to 2D predictions
# # predictions[mask] = ratings[:,advisor][mask] #replace the predictions with the actual ratings where available
# all_predictions.append(predictions) #append predictions


final_prediction_array = np.stack(all_predictions).T #stack the predictions
min_predictions = np.min(final_prediction_array, axis=1) #minimum advisor score (as predicted)
# print(f"Number of valid grids (as predicted): {np.sum((min_predictions > 0.9) & (min_predictions < 1.1))}") #number of valid grids (as predicted)
print(f"Number of valid grids (as predicted): {np.sum(min_predictions > 0.8)}") #number of valid grids (as predicted)
top_100_indices = np.argpartition(min_predictions, -100)[-100:] #indices of top 100 designs (as sorted by minimum advisor score)


top_100_grids = unique_grids[top_100_indices] #get the top 100 grids
print(f"top_100_grids: {top_100_grids[:1]}")

#take the second best 100 from top 200
top_500_indices = np.argpartition(min_predictions, -5000)[-5000:]  # indices of top 200 designs
top_500_indices = top_500_indices[np.argsort(-min_predictions[top_500_indices])] #sort
final_prediction_array[top_500_indices[:100]]
# top_100_grids = unique_grids[top_500_indices[100:200]] #get the top 100 grids
top_500_grids = unique_grids[top_500_indices] #get the top 500 grids

min_predictions_top5 = np.min(top_100_grids, axis=1) #minimum advisor score (as predicted)
# print(top_100_grids.shape)
# print(f"top_100_grids: {top_100_grids[:1]}")
# final_prediction_array[top_100_indices]



In [ ]:
unique_grids
final_prediction_array = np.stack(all_predictions).T #stack the predictions


In [ ]:
min_predictions

In [ ]:
def compute_grid_distance(grid1, grid2):
    """Compute the distance between two grids based on element-wise differences"""
    return np.sum(grid1 != grid2)  # Count different elements

def select_diverse_grids(grids, n_select=100):
    """
    Select n_select most diverse grids from the input grids using a greedy approach.
    Args:
        grids: numpy array of shape (N, 7, 7) containing grid designs
        n_select: number of grids to select
    Returns:
        numpy array of shape (n_select, 7, 7) containing selected diverse grids
    """
    n_grids = len(grids)
    
    # Start with the first grid
    selected_indices = [0]
    remaining_indices = list(range(1, n_grids))
    
    # Greedy selection: at each step, select the grid that is most different from already selected ones
    for _ in range(n_select - 1):
        max_min_distance = -1
        best_idx = -1
        
        # For each remaining grid
        for idx in remaining_indices:
            # Compute minimum distance to any selected grid
            min_distance = float('inf')
            for selected_idx in selected_indices:
                distance = compute_grid_distance(grids[idx], grids[selected_idx])
                min_distance = min(min_distance, distance)
            
            # Update best candidate if this grid has a larger minimum distance
            if min_distance > max_min_distance:
                max_min_distance = min_distance
                best_idx = idx
        
        # Add the best grid to selected and remove from remaining
        selected_indices.append(best_idx)
        remaining_indices.remove(best_idx)
    
    return grids[selected_indices]

# Select 100 most diverse grids from top 500
diverse_grids = select_diverse_grids(top_500_grids, n_select=100)
print(f"Selected {len(diverse_grids)} diverse grids")

# Calculate average pairwise distance to verify diversity
total_distance = 0
n_pairs = 0
for i in range(len(diverse_grids)):
    for j in range(i + 1, len(diverse_grids)):
        total_distance += compute_grid_distance(diverse_grids[i], diverse_grids[j])
        n_pairs += 1

avg_distance = total_distance / n_pairs
print(f"Average pairwise distance between selected grids: {avg_distance:.2f} different elements")

# Show shape of the selected grids
print(f"Shape of selected grids: {diverse_grids.shape}")

# Update top_100_grids with the diverse selection
top_100_grids = diverse_grids

In [ ]:
plot_ratings_histogram(final_prediction_array, withmin=False) #plot histograms of top 100 designs
score = diversity_score(top_100_grids)
print(f"Hypothetical diversity score if all top 100 grids are valid: {score:.4f} ")

final_submission = top_100_grids.astype(int)
assert final_submission.shape == (100, 7, 7)
assert final_submission.dtype == int
assert np.all(np.greater_equal(final_submission, 0) & np.less_equal(final_submission, 4))
id = np.random.randint(1e8, 1e9-1)
np.save(f"results/{id}.npy", final_submission)
print(id)

In [ ]:
score = diversity_score(top_100_grids)
print(f"Hypothetical diversity score if all top 100 grids are valid: {score:.4f} ")

In [ ]:

final_submission = top_100_grids.astype(int)
assert final_submission.shape == (100, 7, 7)
assert final_submission.dtype == int
assert np.all(np.greater_equal(final_submission, 0) & np.less_equal(final_submission, 4))
id = np.random.randint(1e8, 1e9-1)
np.save(f"results/{id}.npy", final_submission)
print(id)

In [ ]:
Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 5
Mean advisor rating (all advisors, all grids): 0.9523465547605141
Score: 0.4956
Submission ID: bc9481fc-aaa9-4a46-87df-798fa1f1c9d2

Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 0
Mean advisor rating (all advisors, all grids): 0.9685816680563087
Score: 0.5281142857142858
Submission ID: 121d0fd3-1b9b-4a9b-97ac-5a6fbeb02cbf

Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 0
Mean advisor rating (all advisors, all grids): 0.9711090626078004
Score: 0.5065795918367347
Submission ID: 13b28d1a-3d76-4bd4-91a7-95cb1516d445

Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 0
Mean advisor rating (all advisors, all grids): 0.9741715630958115
Score: 0.45702448979591836
Submission ID: e62da564-c68e-4f47-86cc-703cfa001fff

Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 1
Mean advisor rating (all advisors, all grids): 0.9572605151740811
Score: 0.5132081632653062
Submission ID: 84fe22d8-d223-4af5-95c8-a07e52e95d28

Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 0
Mean advisor rating (all advisors, all grids): 0.9725441426266319
Score: 0.48675510204081635
Submission ID: 706baab0-4ced-4005-a7d4-0dd37bc1f5c6

Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 12
Mean advisor rating (all advisors, all grids): 0.9528679005486089
Score: 0.41
Submission ID: 66cc4520-cb89-4a13-b051-494a7cd10873


Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 5
Mean advisor rating (all advisors, all grids): 0.9638089872443311
Score: 0.4572857142857143
Submission ID: 22f26771-1ff7-4067-ad75-84a3688d4a63

Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 4
Mean advisor rating (all advisors, all grids): 0.9656329994097232
Score: 0.4459673469387755
Submission ID: ea60ec34-a00c-407b-a6ee-3ab9bb793c46


---
top 100
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 5
Mean advisor rating (all advisors, all grids): 0.9596925926849548
Score: 0.40858367346938773
Submission ID: 61199621-5c5c-4f38-9122-f87615f8cc8e

top second 100
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 11
Mean advisor rating (all advisors, all grids): 0.9523805852710479
Score: 0.3774
Submission ID: 2254a970-e1ca-4bd5-9656-f13a36f728c5


top third 100
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 9
Mean advisor rating (all advisors, all grids): 0.9462318970556141
Score: 0.4081102040816327
Submission ID: be449510-b190-4fae-9387-16f19a83f4f7

4
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 18
Mean advisor rating (all advisors, all grids): 0.9378541694451326
Score: 0.33855510204081635
Submission ID: 76ad3233-0c75-45db-abb0-a27624250e0c

5
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 16
Mean advisor rating (all advisors, all grids): 0.9419590764106018
Score: 0.34891836734693876
Submission ID: 5d532188-06a8-4755-a2f1-adfec25858e2

after the selection of diverse grids
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 18
Mean advisor rating (all advisors, all grids): 0.9290512422197501
Score: 0.3576530612244898
Submission ID: 7cd8bcb4-52ac-4e53-b9d6-4523c52e22ba

out of top 1000
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 26
Mean advisor rating (all advisors, all grids): 0.9074144134730059
Score: 0.29511020408163263
Submission ID: 715da021-fee3-479c-8d89-965bbf283da7

out of 200
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 9
Mean advisor rating (all advisors, all grids): 0.9516056723356537
Score: 0.4150122448979592
Submission ID: 979efc73-bc95-45ac-a7e6-8f88f74cbcda


out of 200
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 6
Mean advisor rating (all advisors, all grids): 0.9580031518265091
Score: 0.4419265306122449
Submission ID: 7dc28e59-0b49-46e7-a0e0-27fbea16fe53

best 100 after generation 1 different
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 3
Mean advisor rating (all advisors, all grids): 0.9661947331155596
Score: 0.45669795918367345
Submission ID: 07d7122d-c2f4-47f1-b9d8-23782773fd83

second 100 after generation 1 different
Submission Successful!
Name: Patryk&Angela
Nickname: Patryk&Angela
# Invalid grids: 7
Mean advisor rating (all advisors, all grids): 0.9571057667558083
Score: 0.4366204081632653
Submission ID: 278bcd2a-561b-499f-bccc-e0c1ca8a24a4

Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 8
Mean advisor rating (all advisors, all grids): 0.9563033659106659
Score: 0.44416326530612243
Submission ID: 1efa721d-2ca2-4e05-b7cc-3d5b98162b2d


Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 1
Mean advisor rating (all advisors, all grids): 0.9645268877404828
Score: 0.49383673469387757
Submission ID: 5689d589-d598-44ce-a1d4-adb0d9c0fd90

with 2 different cells
Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 9
Mean advisor rating (all advisors, all grids): 0.9459172273977517
Score: 0.43793061224489793
Submission ID: cad1525d-81c5-4deb-a5ed-a8bbe76008fd

Submission Successful!
Name: P&A
Nickname: P&A
# Invalid grids: 7
Mean advisor rating (all advisors, all grids): 0.9532169353250773
Score: 0.41968163265306124
Submission ID: 37d0a701-a4d3-499f-bfd5-6414626507db

In [ ]:
final_prediction_array

In [ ]:
from save_load_utils import save_prediction_data

# Save the current data
save_prediction_data(unique_grids, final_prediction_array)

# Example of how to load it back (commented out as we already have the data in memory)
# unique_grids_loaded, final_prediction_array_loaded = load_prediction_data()
print("Data saved successfully to the 'saved_data' directory.")

In [ ]:
from save_load_utils import load_prediction_data
loaded_grids, loaded_predictions = load_prediction_data()